### Initialize Workspace

In [1]:
import analysis_parameters
import xarray as xr
import glob
import numpy as np
import matplotlib.pyplot as plt
import multimodelstats as mms

In [2]:
data_path = analysis_parameters.DIR_INTERMEDIATE_DATA

### Get relevant filenames and datasets

In [3]:
def get_scenario_fnames(data_path, scenario):
    """
    Get a string list of all zarr files in the data_path for the given 
    scenario. Prints the list to the user.
    """
    endcut = -1*len('.zarr')
    begcut = len(data_path)
    names = [f[begcut:endcut] for f in glob.glob(data_path + '*_' + scenario + '_*.zarr')]
    return names

def read_in_fname(data_path, fname):
    """Read in zarr file with name datapath/fname.zarr and return the correpsonding xarray"""
    filename = data_path + fname + '.zarr'
    return xr.open_zarr(filename)

In [4]:
scenario_name = 'historical'
file_names_hist = get_scenario_fnames(data_path, scenario_name)
datasets_hist = [read_in_fname(data_path, fname) for fname in file_names_hist]

### Initialize Dataset

Currently a hacky way to make empty datasets of the right dimensions

In [5]:
initial_ds_fname = 'tas_historical_CAMS-CSM1-0'
variable_name = 'tas'
#def initialize_dataset(data_path, initial_ds_fname):
ds_init = read_in_fname(data_path, initial_ds_fname)
ds_init.load()

ds_dims = np.shape(ds_init[variable_name].values)
lats = ds_init['lat'].values
lons = ds_init['lon'].values
times = ds_init['time']
empty_array = np.empty(ds_dims)

In [6]:
mean_vals = np.copy(empty_array)
max_vals = np.copy(empty_array)
min_vals = np.copy(empty_array)
std_vals = np.copy(empty_array)

### Loop through lat/lons and calculate for each

***This currently takes ~7-8 mins to run for each latitude band (160 total), which means it will take about 21 hours to run for all***

In [ ]:
import time
start_time = time.time()
for i in range(0,len(lats)):
    lt = lats[i]
    print(i)
    for j in range(0,len(lons)):
        ln = lons[j]
        stats_single_point = mms.export_stats(datasets_hist, file_names_hist, is_global_mean=False, coords=[lt, ln])
        mean_vals[:,i,j] = stats_single_point['multi_mean'].values
        min_vals[:,i,j] = stats_single_point['multi_min'].values
        max_vals[:,i,j] = stats_single_point['multi_max'].values
        std_vals[:,i,j] = stats_single_point['multi_std'].values
    end_time = time.time()
    print(end_time - start_time)

0
472.96125960350037
1
915.7842659950256
2
1367.3400619029999
3


In [ ]:
# if you have to stop running this partway through, save your progress to pickle
output_path_temp = '/home/jovyan/local-climate-data-tool/Data/IntermediateData/'
fname_temp = 'temporary_multi_model_stats_grid'

with open(output_path_temp+fname_temp+'mean'+'.pickle', 'wb') as handle:
    pickle.dump(mean_vals, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(output_path_temp+fname_temp+'min'+'.pickle', 'wb') as handle:
    pickle.dump(min_vals, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(output_path_temp+fname_temp+'max'+'.pickle', 'wb') as handle:
    pickle.dump(max_vals, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(output_path_temp+fname_temp+'std'+'.pickle', 'wb') as handle:
    pickle.dump(std_vals, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Started running at 11:15 am, done with 0-1 by 11:25 am
print(i)

### Check memory usage

In [7]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('empty_array', 811008128),
 ('max_vals', 811008128),
 ('mean_vals', 811008128),
 ('min_vals', 811008128),
 ('std_vals', 811008128),
 ('datasets_hist', 264),
 ('file_names_hist', 264),
 ('get_scenario_fnames', 136),
 ('read_in_fname', 136),
 ('ds_init', 112),
 ('data_path', 108),
 ('lats', 96),
 ('lons', 96),
 ('times', 96),
 ('mms', 80),
 ('np', 80),
 ('plt', 80),
 ('xr', 80),
 ('initial_ds_fname', 75),
 ('ds_dims', 72),
 ('scenario_name', 59),
 ('variable_name', 52)]

### Save Dataset

In [8]:
# Create xarrays from numpy array
ds_coords = {'time': ds_init['time'], 'lat': ds_init['lat'], 'lon': ds_init['lon']}
ds_dims = ('time', 'lat', 'lon')
mean_xr = xr.DataArray(mean_vals, dims=ds_dims, coords=ds_coords)

max_xr = xr.DataArray(max_vals, dims=ds_dims, coords=ds_coords)

min_xr = xr.DataArray(min_vals, dims=ds_dims, coords=ds_coords)

std_xr = xr.DataArray(std_vals, dims=ds_dims, coords=ds_coords)

# Create xarray dataset from xarrays
ds = xr.Dataset({'mean': mean_xr,
                 'min': min_xr,
                 'max': max_xr,
                 'std': std_xr,},
                coords=ds_coords)

In [ ]:
#save dataset to zarr
def export_dataset(ds, output_path, variable_name, scenario_name):
    ds.chunk({'lon':10, 'lat':10, 'time':-1})
    ds.to_zarr(output_path+'modelData_'+variable_name+'_'+scenario_name+'.zarr')

export_dataset(ds=DS,
               output_path='/home/jovyan/local-climate-data-tool/Data/ProcessedData/',
               variable_name=variable_name,
               scenario_name=scenario_name)